## NOTEBOOK 10

In this  notebook we are going to integrate the genetic algorithm of the previous lessonin order to integrate a way:

- to perform a simulated annealing search of the best path
- to use MPI library to perform more than one search of the best path in parallel

### EXERCISE 10.1